In [1]:
import pickle
import pandas as pd
import numpy as np
import torchvision
from torch.utils.data import DataLoader, Subset
import torchvision.transforms as transforms
import torch
from sklearn.svm import SVC

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [3]:
"""
def load_data(path, label):
    with open(path, 'rb') as iffile:
        features = pickle.load(iffile)
    layer = 'view'
    features = features[layer]
    features_matrix = None

    for i in range(len(features)):
        if isinstance(features_matrix, torch.Tensor):
            features_matrix = torch.cat((   features_matrix, features[i]))
        else:
            features_matrix = features[i]

    features = None    
    features_matrix = torch.flatten(features_matrix, start_dim=1, end_dim=-1)
    y = np.empty(features_matrix.shape[0])
    y.fill(label)
    return [features_matrix, y]
"""

def load_data(path, label):
    features_matrix = torch.load(path)
    y = np.empty(features_matrix.shape[0])
    y.fill(label)
    return [features_matrix, y]

def unison_shuffled_copies_ind(X, y):
    assert len(X) == len(y)
    p = np.random.permutation(len(X))
    return X[p], y[p]


def unison_shuffled_copies(x_ben, y_ben, x_adv, y_adv):
    X = np.concatenate((x_ben, x_adv), axis=0)
    y = np.concatenate((y_ben, y_adv), axis=0)
    assert len(X) == len(y)
    p = np.random.permutation(len(X))
    return X[p], y[p]


def get_paths(base_path):
    attack="CW"
    total_attack_samples = 5120
    integrated = False
    relu_dir = "ReLUs"
    detector_type='Regular'
    train_benign = f"ReLUs_{attack}_train_benign_{total_attack_samples}_detector-type-{detector_type}_integrated-{integrated}.pth"
    train_adversarial = f"ReLUs_{attack}_train_adversarial_{total_attack_samples}_detector-type-{detector_type}_integrated-{integrated}.pth"
    test_benign = f"ReLUs_{attack}_test_benign_{total_attack_samples}_detector-type-{detector_type}_integrated-{integrated}.pth"
    test_adversarial = f"ReLUs_{attack}_test_adversarial_{total_attack_samples}_detector-type-{detector_type}_integrated-{integrated}.pth"

    train_benign = os.path.join(base_path, relu_dir, train_benign)
    train_adversarial = os.path.join(base_path, relu_dir,  train_adversarial)
    test_benign = os.path.join(base_path,  relu_dir, test_benign)
    test_adversarial = os.path.join(base_path,  relu_dir, test_adversarial)

    all_paths = [train_benign,train_adversarial,test_benign,test_adversarial]

    return all_paths




In [4]:
def quantize(X):
    X[0] = torch.where(X[0] > 0, torch.ones_like(X[0]), torch.zeros_like(X[0]))
    return X

In [5]:
expr_dir = "/bigstor/zsarwar/SparseDNNs/MT_CIFAR10_full_10_d5f3f545a0883adb9c8f98e2a6ba4ac7/MT_Baseline_32efd89f04333501e1d66bc4e1d4260f"

train_benign,train_adversarial,test_benign,test_adversarial = get_paths(expr_dir)
train_benign = load_data(train_benign, 1)
train_adversarial = load_data(train_adversarial, -1)
test_benign = load_data(test_benign, 1)
test_adversarial = load_data(test_adversarial, -1)

"""
train_benign = quantize(train_benign)
train_adversarial = quantize(train_adversarial)
test_benign = quantize(test_benign)
test_adversarial = quantize(test_adversarial)
"""

'\ntrain_benign = quantize(train_benign)\ntrain_adversarial = quantize(train_adversarial)\ntest_benign = quantize(test_benign)\ntest_adversarial = quantize(test_adversarial)\n'

In [6]:
X_train, y_train = unison_shuffled_copies(train_benign[0], train_benign[1], train_adversarial[0], train_adversarial[1])

X_test, y_test = unison_shuffled_copies(test_benign[0], test_benign[1], test_adversarial[0], test_adversarial[1])



In [48]:
clf = SVC(C=0.9, gamma=0.4)
clf.fit(X_train, y_train)


SVC(C=0.9, gamma=0.4)

In [49]:
# 0.075

# Testing
#x_train_pred = clf.predict(X_train)
#train_acc = len(np.where(x_train_pred == y_train)[0]) / len(X_train)

#x_test_pred = clf.predict(X_test)
#test_acc = len(np.where(x_test_pred == y_test)[0]) / len(X_test)



#print(f"Training acc : {train_acc}")
#print(f"Test acc : {test_acc}")

# Test classwise
#x_train_ben_pred = clf.predict(train_benign[0])
#train_ben_acc = len(np.where(x_train_ben_pred == train_benign[1])[0]) / len(x_train_ben_pred)

#x_train_adv_pred = clf.predict(train_adversarial[0])
#train_adv_acc = len(np.where(x_train_adv_pred == train_adversarial[1])[0]) / len(x_train_adv_pred)

x_test_ben_pred = clf.predict(test_benign[0])
test_ben_acc = len(np.where(x_test_ben_pred == test_benign[1])[0]) / len(x_test_ben_pred)

x_test_adv_pred = clf.predict(test_adversarial[0])
test_adv_acc = len(np.where(x_test_adv_pred == test_adversarial[1])[0]) / len(x_test_adv_pred)

#print(f"Training benign acc : {train_ben_acc}")
#print(f"Training adversarial acc : {train_adv_acc}")
print(f"-------------------------------------")
print(f"Testing benign acc : {test_ben_acc}")
print(f"Testing adversarial acc : {test_adv_acc}")



-------------------------------------
Testing benign acc : 0.830859375
Testing adversarial acc : 0.703515625


In [46]:
# 0.005

# Testing
x_train_pred = clf.predict(X_train)
train_acc = len(np.where(x_train_pred == y_train)[0]) / len(X_train)

x_test_pred = clf.predict(X_test)
test_acc = len(np.where(x_test_pred == y_test)[0]) / len(X_test)



print(f"Training acc : {train_acc}")
print(f"Test acc : {test_acc}")

# Test classwise
x_train_ben_pred = clf.predict(train_benign[0])
train_ben_acc = len(np.where(x_train_ben_pred == train_benign[1])[0]) / len(x_train_ben_pred)

x_train_adv_pred = clf.predict(train_adversarial[0])
train_adv_acc = len(np.where(x_train_adv_pred == train_adversarial[1])[0]) / len(x_train_adv_pred)

x_test_ben_pred = clf.predict(test_benign[0])
test_ben_acc = len(np.where(x_test_ben_pred == test_benign[1])[0]) / len(x_test_ben_pred)

x_test_adv_pred = clf.predict(test_adversarial[0])
test_adv_acc = len(np.where(x_test_adv_pred == test_adversarial[1])[0]) / len(x_test_adv_pred)

print(f"Training benign acc : {train_ben_acc}")
print(f"Training adversarial acc : {train_adv_acc}")
print(f"-------------------------------------")
print(f"Testing benign acc : {test_ben_acc}")
print(f"Testing adversarial acc : {test_adv_acc}")



Training acc : 0.9544921875
Test acc : 0.8935546875
Training benign acc : 0.9654296875
Training adversarial acc : 0.9435546875
-------------------------------------
Testing benign acc : 0.8552734375
Testing adversarial acc : 0.9318359375


In [27]:
# Quantize
# Comput per-dimension mean

In [ ]:
dim_mean = features_matrix.mean(dim=0)

In [ ]:
binarized_matrix = torch.where(features_matrix >= dim_mean, torch.ones_like(features_matrix), torch.zeros_like(features_matrix))

In [ ]:
features_matrix = None

In [ ]:

torch.cuda.empty_cache()

In [ ]:
binarized_matrix = binarized_matrix.cpu().numpy()